In [1]:
season = {
    "bundes_22-23":[ 1736824, 1737122,298 ],
    "Prem_Leag_22-23":[1736219,1736589,370],
    "Premiere_22-23":[1744303,1744668,365],
    "Series_A_22-23":[1747394,1747768,374]
}

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep

data_list = list()
df = pd.read_csv('data/table_matches_new.csv')
df = df.drop(columns=['Unnamed: 0'])
count = 1736543
        
while count != 1736589 :
    count += 1
    
    url_1 = f"https://soccer365.ru/games/{count}/"
    
    url_2 = f"https://soccer365.ru/games/{count}/&tab=form_teams"
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 YaBrowser/23.7.0.2534 Yowser/2.5 Safari/537.36'
    }
    response_1 = requests.get(url_1,headers=headers)
    sleep(10)
    response_2 = requests.get(url_2,headers=headers)
    sleep(9)

    soup_1 = BeautifulSoup(response_1.text,'lxml')
    soup_2 = BeautifulSoup(response_2.text,'lxml')
    
    title = ['id',"общая информация","Дата",'Команда1',"счет1",'Команда2',"счет2",'Температура',"Признак погоды","поб1","X","поб2","ТМ 2.5","ТБ 2.5"]
    title_Summary_statistics = ['Матчи 1','Матчи 2','Отдых: дни (часы) 1','Отдых: дни (часы) 2','Победы 1','Победы 2','Ничьи 1','Ничьи 2','Поражения 1','Поражения 2','Забито голов 1','Забито голов 2','Пропущено голов 1','Пропущено голов 2','Забито голов за игру 1','Забито голов за игру 2','Пропущено голов за игру 1','Пропущено голов за игру 2','Сухие матчи 1','Сухие матчи 2','обе забьют 1','обе забьют 2','Тотал больше 2.5 1','Тотал больше 2.5 2','Тотал меньше 2.5 1','Тотал меньше 2.5 2','Удары 1','Удары 2','Удары  (соперник) 1','Удары  (соперник) 2','Удары в створ 1','Удары в створ 2','Удары в створ  (соперник) 1','Удары в створ  (соперник) 2','Владение 1','Владение 2','Владение  (соперник) 1','Владение  (соперник) 2','Угловые 1','Угловые 2','Угловые  (соперник) 1','Угловые  (соперник) 2','Нарушения 1','Нарушения 2','Нарушения  (соперник) 1','Нарушения  (соперник) 2','Офсайды 1','Офсайды 2','Офсайды  (соперник) 1','Офсайды  (соперник) 2','Желтые карточки 1','Желтые карточки 2','Желтые карточки  (соперник) 1','Желтые карточки  (соперник) 2','Красные карточки 1','Красные карточки 2','Красные карточки  (соперник) 1','Красные карточки  (соперник) 2']
    full_title = title + title_Summary_statistics
    
    lister = list()
    lister.append(count)
    
    #Общая инфа и время матча
    data = soup_1.find("div",class_="block_header bkcenter").text.replace("\n","")
    date_list = data.split()
    date_time = date_list[-2] + ' ' + date_list[-1]
    lister.append(data)
    lister.append(date_time)

    #Название команд и счет

    result_match_1= soup_1.find("div",class_="live_game left")
    result_match_2= soup_1.find("div",class_="live_game right")

    name_1_com = result_match_1.find("div",class_="live_game_ht").text.replace("\n","")
    goal_1_com = result_match_1.find("div",class_="live_game_goal").text.replace("\n","")

    name_2_com = result_match_2.find("div",class_="live_game_at").text.replace("\n","")
    goal_2_com = result_match_2.find("div",class_="live_game_goal").text.replace("\n","")
    lister.append(name_1_com)
    lister.append(goal_1_com)
    lister.append(name_2_com)
    lister.append(goal_2_com)


    #Погода
    preview_st = soup_1.find("div",class_="preview_item st")
    temper = preview_st.find("span",class_="red").text

    priznac_pog = preview_st.find_all("span",class_="min_gray")[1].text
    lister.append(temper)
    lister.append(priznac_pog)


    #Коэфиценты
    coeficent_all = soup_1.find_all("div",class_="block_body_nopadding")[3]
    coficient = coeficent_all.find_all('div',class_='odds_item odds_logo')

    def cof_func(coficient):
        list_coef = list()
        for i in coficient:
            odds_coeff = i.find_all('div',class_='odds_coeff')
            for j in odds_coeff:
                if j == '\n':
                    continue
                if j == '\xa0':
                    list_coef.clear()
                    break
                list_coef.append(float(j.text))
            if len(list_coef) == 5:
                return  list_coef  
            
    coficient_result = cof_func(coficient)
    lister.extend(coficient_result)
    result_matches_to10 = soup_2.find_all("div", class_="block_body_nopadding")
    Summary_statistics = result_matches_to10[3] 
    table_tr_1 = Summary_statistics.find_all("td")
    Summary_statistics2 = result_matches_to10[4] 
    table_tr_2 = Summary_statistics2.find_all("td")
    
    def reformed_tabl_1(table):
        lens = len(table)
        col_priz = lens / 3
        lister_pr = list()
        lister_res = list()
        for i in range(0,lens):
            if i in [1,4,7,10,13,16,19,22,25,28,31,34,37]:
                lister_pr.append(table[i].text + "1")
                lister_pr.append(table[i].text + "2")
            else:
                lister_res.append(table[i].text.replace("\xa0"," "))
        return(lister_pr,lister_res)

    test_tr1,test_res1 = reformed_tabl_1(table_tr_1)

    #display(test_tr)
    #display(test_res)

    def reformed_tabl_2(table):
        lens = len(table)
        col_priz = lens / 3
        lister_pr = list()
        lister_res = list()
        for i in range(0,lens):
            if i in [1,4,7,10,13,16,19,22,25,28,31,34,37,40,43,46]:
                lister_pr.append(table[i].text + "1")
                lister_pr.append(table[i].text + "2")
            else:
                lister_res.append(table[i].text.replace("\xa0"," "))
        return(lister_pr,lister_res)

    test_tr2,test_res2 = reformed_tabl_2(table_tr_2)

    #display(test_tr)
    #display(test_res)
    final_test_tr = test_tr1 + test_tr2
    final_test_res = test_res1 + test_res2
    lister.extend(final_test_res)
    data_list.append(lister)
    

IndexError: list index out of range

In [ ]:
lister

[1736544,
 'Премьер-Лига, 4-й тур, регулярный сезон, 03.09.2023 16:00',
 '03.09.2023 16:00',
 'Кристал Пэлас',
 '-',
 'Вулверхэмптон',
 '-',
 '+17°C',
 'небольшой дождь']

In [ ]:
res_df = pd.DataFrame(
    data = data_list,
    columns= full_title,
)

In [ ]:
res_df

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
0,1736335,"Премьер-Лига, 12-й тур, регулярный сезон, 18.1...",18.10.2022 22:15,Кристал Пэлас,2,Вулверхэмптон,1,+17°C,небольшой дождь,2.09,...,1.40,1.20,1.90,1.90,2.40,1.80,0,0.10,0.10,0
1,1736336,"Премьер-Лига, 12-й тур, регулярный сезон, 19.1...",19.10.2022 21:30,Ньюкасл,1,Эвертон,0,+17°C,переменная облачность,1.66,...,2.20,1.20,1.50,2.30,1.60,2.10,0.10,0,0.10,0
2,1736337,"Премьер-Лига, 12-й тур, регулярный сезон, 19.1...",19.10.2022 21:30,Ливерпуль,1,Вест Хэм,0,+17°C,пасмурно,1.44,...,2.30,1.00,1.30,2.00,1.20,2.50,0,0,0,0
3,1736338,"Премьер-Лига, 12-й тур, регулярный сезон, 19.1...",19.10.2022 22:15,Ман Юнайтед,2,Тоттенхэм,0,+17°C,умеренный или сильный ливневый дождь,2.44,...,0.50,0.70,2.00,2.40,1.70,2.30,0,0.10,0,0.20
4,1736339,"Премьер-Лига, 13-й тур, регулярный сезон, 23.1...",23.10.2022 16:00,Астон Вилла,4,Брентфорд,0,+17°C,переменная облачность,2.08,...,0.70,1.50,1.90,1.50,1.70,1.40,0.10,0,0.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,1736539,"Премьер-Лига, 33-й тур, регулярный сезон, 27.0...",27.04.2023 21:45,Эвертон,1,Ньюкасл,4,+17°C,пасмурно,4.40,...,1.40,0.40,2.10,1.40,1.60,2.10,0.20,0.10,0.10,0
205,1736540,"Премьер-Лига, 33-й тур, регулярный сезон, 25.0...",25.04.2023 22:00,Лидс,1,Лестер Сити,1,+16°C,переменная облачность,2.33,...,2.80,2.40,1.80,2.00,1.70,2.10,0,0.20,0.20,0.10
206,1736541,"Премьер-Лига, 33-й тур, регулярный сезон, 26.0...",26.04.2023 21:30,Ноттингем Форест,3,Брайтон,1,+17°C,переменная облачность,5.35,...,1.20,1.60,2.40,0.90,2.00,1.80,0,0,0,0
207,1736542,"Премьер-Лига, 33-й тур, регулярный сезон, 27.0...",27.04.2023 22:15,Тоттенхэм,2,Ман Юнайтед,2,+17°C,небольшой дождь,2.82,...,1.70,1.20,2.30,1.80,1.20,1.30,0.20,0.10,0.10,0.20


In [ ]:
res_df1
#res_df1.to_csv('data/table_res_math_Prem_Leag_220-334.csv')

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
0,1736220,"Премьер-Лига, 1-й тур, регулярный сезон, 06.08...",06.08.2022 14:30,Фулхэм,2,Ливерпуль,2,+17°C,небольшой дождь,11.00,...,2.83,2.20,0.78,0.40,0.44,0.60,0,0,0,0
1,1736221,"Премьер-Лига, 1-й тур, регулярный сезон, 06.08...",06.08.2022 17:00,Борнмут,2,Астон Вилла,0,+18°C,переменная облачность,3.95,...,3.17,1.00,0.38,1.22,0.25,1.11,0,0,0,0.11
2,1736222,"Премьер-Лига, 1-й тур, регулярный сезон, 06.08...",06.08.2022 17:00,Лидс,2,Вулверхэмптон,1,+16°C,переменная облачность,2.41,...,1.20,1.50,1.33,1.71,2.00,1.86,0.22,0,0.11,0
3,1736223,"Премьер-Лига, 1-й тур, регулярный сезон, 07.08...",07.08.2022 16:00,Лестер Сити,2,Брентфорд,2,+17°C,переменная облачность,1.91,...,1.75,2.00,0.38,1.00,1.63,0.60,0,0.20,0,0.40
4,1736224,"Премьер-Лига, 1-й тур, регулярный сезон, 06.08...",06.08.2022 17:00,Ньюкасл,2,Ноттингем Форест,0,+17°C,переменная облачность,1.58,...,2.00,1.00,1.11,0.57,1.22,0.43,0.11,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,1736330,"Премьер-Лига, 12-й тур, регулярный сезон, 19.1...",19.10.2022 21:30,Борнмут,0,Саутгемптон,1,+18°C,переменная облачность,2.69,...,2.10,0.90,1.40,1.60,1.30,1.90,0,0,0,0
111,1736331,"Премьер-Лига, 12-й тур, регулярный сезон, 19.1...",19.10.2022 21:30,Брентфорд,0,Челси,0,+18°C,умеренный дождь,4.33,...,1.70,1.10,1.50,2.30,1.80,2.10,0,0.10,0,0.10
112,1736332,"Премьер-Лига, 12-й тур, регулярный сезон, 18.1...",18.10.2022 21:30,Брайтон,0,Ноттингем Форест,0,+18°C,пасмурно,1.39,...,2.00,2.40,1.10,2.70,2.40,1.70,0,0,0,0
113,1736333,"Премьер-Лига, 12-й тур, регулярный сезон, 20.1...",20.10.2022 21:30,Фулхэм,3,Астон Вилла,0,+17°C,небольшой дождь,2.97,...,2.80,1.00,2.70,2.10,1.50,1.80,0.10,0,0,0.10


In [ ]:
#a = pd.concat([res_df1,res_df])
#a.to_csv('data/table_res_math_Prem_Leag_22-23.csv')

In [ ]:
543